# Descrição da Classe AutomatosFinitos

A classe `AutomatosFinitos` implementa um automato finito determinístico (AFD) com base num padrão de procura, permitindo encontrar todas as ocorrências desse padrão dentro de um texto, além de visualizar a tabela de transições e o caminho de estados percorridos.

---

## Projeto de Alto Nível (AutomatosFinitos)

### 1. **Entrada e Saída**:
- **Entrada:**
    - Um alfabeto (set de símbolos).
    - Um padrão (str) a ser reconhecido no texto.
- **Saída:**
    - Lista de estados percorridos (process_input).
    - Lista de posições de ocorrências (find_occurrences).
    - Impressão da tabela de transição (visualize).

### 2. **Componentes Principais**:
- ` __init__`:
    - Inicializa o automato com o alfabeto e o padrão.
    - Cria a tabela de transições.
- `_build_transition_table`:
    - Constrói a tabela de transições entre estados com base no padrão.
- ` _calculate_state`:`
    - Calcula o próximo estado ideal dado um símbolo e o estado atual, considerando sobreposição máxima com o início do padrão.
- `process_input`:
    - Percorre a sequência de entrada, seguindo as transições do automato, e retorna os estados visitados.
- `find_occurrences`:
    - Encontra todas as posições no texto onde o padrão ocorre completamente.
- `visualize`:
    - Imprime a estrutura do automato (estados e transições).
- `get_next_state`:
    - Retorna o próximo estado, validando o símbolo contra o alfabeto.
---
## Projeto de Baixo Nível 

### 1. **Construção da Tabela de Transição**:
- **Descrição:**
    - Para cada estado e cada símbolo do alfabeto, define o próximo estado com base na maior sobreposição entre o sufixo atual e o prefixo do padrão.
- **Algoritmo:**
    - Itera sobre todos os pares (estado, símbolo).
    - Usa `_calculate_state` para determinar o estado de destino.


### 2. **Cálculo do Estado**:
- **Descrição:**
    - Dado um estado atual e símbolo, calcula o próximo estado simulando a adição do símbolo ao sufixo do padrão visto até o momento.
- **Algoritmo:**
    - Junta o prefixo do padrão ao estado atual com o novo símbolo.
    - Verifica a maior sobreposição com o início do padrão.


### 3. **Processamento da Entrada**:
- **Descrição:**
    - Simula a execução do automato sobre uma sequência.
- **Algoritmo:**
    - Começa do estado 0.
    - Para cada caractere da sequência, aplica a transição e armazena o estado visitado.


### 4. **Busca de Ocorrências**:
- **Descrição:**
    - Identifica todos os pontos do texto onde o padrão foi completamente reconhecido.
- **Algoritmo:**
    - Executa `process_input`.
    - Se o estado final de algum ponto for igual ao número de caracteres do padrão, significa que houve uma ocorrência.


### 5. **Visualização**:
- **Descrição:**
    - Imprime todos os estados e suas transições de forma organizada.
- **Algoritmo:**
    - Itera sobre todos os pares (estado, símbolo).
    - Usa _calculate_state para determinar o estado de destino.

## 6. **Transição segura**
- **Descrição:**
    - Obter com segurança o próximo estado com base no símbolo lido.
- **Algoritmo:**
    - Verifica se o símbolo está no alfabeto (evita erros).
    - Retorna o próximo estado com base na tabela de transição ou 0.

## 7. **Cálculo de Sobreposição entre Padrões** 
- **Descrição:**
    - Calcular o maior "overlap" entre o final de uma string e o início do padrão.
- **Algoritmo:**
    - Compara sufixos do `suffix_candidate` com prefixos do `prefix_source`.
    - Quanto maior o valor retornado, mais "longo" é o trecho que pode continuar o reconhecimento do padrão.

---


## Implementação




In [ ]:
class AutomatosFinitos :

    def __init__(self, alphabet: set, pattern: str):
        if not pattern:
            raise ValueError ('Pattern cannot be empty')
        if not alphabet:
            raise ValueError ('Alphabet cannot be empty')
        
        self.num_states = len(pattern) + 1
        self.alphabet = alphabet
        self.pattern = pattern
        self.transition_table = {}
        self._build_transition_table()

    def _build_transition_table(self):

        """Constructs the transition table using optimized overlap calculation."""

        for state in range(self.num_states):
            for symbol in self.alphabet:
                next_state = self._calculate_state(state, symbol)
                self.transition_table[(state, symbol)] = next_state

    def _calculate_state(self, current_state: int, symbol: str):
        """Calculates the next state with memoization for repeated patterns."""

        if current_state == 0 and symbol == self.pattern[0]:
            return 1
        
        candidate = self.pattern[:current_state] +  symbol
        max_overlap = min(current_state + 1, len(self.pattern))

        for length in range(max_overlap, 0, -1):
            if candidate.endswith(self.pattern[:length]):
                return length
        return 0
    
    def process_input(self, sequence: str) :
        """Processes an input sequence and returns states visited."""
        
        current_state = 0
        states_log = [current_state]

        for char in sequence:
            current_state = self.transition_table.get(
                (current_state, char), 0
            )
            states_log.append(current_state)
        return states_log
    
    def find_occurrences (self, text: str):
        """Finds all pattern occurrences with their starting positions."""

        states = self.process_input(text)
        pattern_length = len(self.pattern)
        matches = [
            index - pattern_length 
            for index, state in enumerate(states)
            if state == len (self.pattern)
        ]
        return len (matches), matches
    
    def visualize (self):
        print(f"States: {self.num_states}")
        print(f"Search Pattern: {self.pattern}")
        print("Transition Map:")
        for state in sorted(set(s for s, _ in self.transition_table)):
            symbols = [k[1] for k in self.transition_table if k[0] == state]
            for symbol in sorted(symbols):
                dest = self.transition_table[(state, symbol)]
                print(f"  ({state}, {symbol}) → {dest}")

    def get_next_state(self, current_state: int, symbol: str):
        """Safe state transition with input validation."""
        if symbol not in self.alphabet:
            raise ValueError(f"Symbol '{symbol}' not in alphabet")
        return self.transition_table.get((current_state, symbol), 0)
    
def pattern_overlap(suffix_candidate: str, prefix_source: str):
    """Calculates maximum overlap between suffix and prefix."""
    max_length = min(len(suffix_candidate), len(prefix_source))
    for length in range(max_length, 0, -1):
        if suffix_candidate[-length:] == prefix_source[:length]:
            return length
    return 0

In [ ]:
alphabet = {'a', 'b', 'c'}
pattern = "abc"

# Create the finite automaton
automaton = AutomatosFinitos(alphabet, pattern)

# Visualize the automaton's transition table
print("Visualizing the automaton:")
automaton.visualize()

# Define a text to search for the pattern
text = "abcabcababc"

# Find occurrences of the pattern in the text
count, positions = automaton.find_occurrences(text)

# Output results
print("\nResults:")
print(f"Text: {text}")
print(f"Pattern: {pattern}")
print(f"Number of occurrences: {count}")
print(f"Starting positions of occurrences: {positions}")

# Process the input sequence and show states visited
states_visited = automaton.process_input(text)
print(f"\nStates visited during processing: {states_visited}")

Visualizing the automaton:
States: 4
Search Pattern: abc
Transition Map:
  (0, a) → 1
  (0, b) → 0
  (0, c) → 0
  (1, a) → 1
  (1, b) → 2
  (1, c) → 0
  (2, a) → 1
  (2, b) → 0
  (2, c) → 3
  (3, a) → 1
  (3, b) → 0
  (3, c) → 0

Results:
Text: abcabcababc
Pattern: abc
Number of occurrences: 3
Starting positions of occurrences: [0, 3, 8]

States visited during processing: [0, 1, 2, 3, 1, 2, 3, 1, 2, 1, 2, 3]


## Projeto de Alto Nível – Burrows-Wheeler Transform (BWT)

### **Projeto de Alto Nível**

1. **Entrada e Saída**:
   - **Entrada**: Uma sequência de caracteres (como DNA ou texto), sem o símbolo de fim.
   - **Saída**: Uma lista com:
     - A sequência original;
     - A sequência transformada pela BWT;
     - A posição da sequência original na matriz de rotações ordenadas.

2. **Componentes Principais**:
   - **Adição do Terminador**: Acrescenta um símbolo especial (`$`) ao final da sequência para garantir unicidade.
   - **Geração das Rotações**: Cria todas as rotações circulares possíveis da sequência com o terminador.
   - **Ordenação Lexicográfica**: Ordena as rotações em ordem alfabética.
   - **Construção da BWT**: Extrai o último caractere de cada rotação ordenada para formar a sequência BWT.
   - **Identificação da Posição Original**: Determina a posição da sequência original dentro da lista ordenada.

3. **Fluxo de Dados**:
   Entrada da sequência → Adição de `$` → Geração das rotações → Ordenação → Extração da última coluna → Localização do índice original → Saída final em lista `[original, bwt, índice]`.

---

### **Projeto de Baixo Nível**

1. **Adição de Terminador (`$`)**:
   - **Descrição**: Adiciona um caractere especial ao final da sequência para garantir que nenhuma outra rotação seja idêntica à original. Isso também facilita a reconstrução posterior da sequência.

2. **Geração de Rotações Cíclicas**:
   - **Descrição**: Cria todas as possíveis rotações da sequência, deslocando os caracteres circularmente. Cada rotação representa uma possível reorganização da sequência.

3. **Ordenação das Rotações**:
   - **Descrição**: Ordena as rotações em ordem lexicográfica (alfabética), o que é essencial para a formação da BWT e permite compactação e busca mais eficiente.

4. **Construção da Sequência BWT**:
   - **Descrição**: Forma a sequência BWT pegando o último caractere de cada rotação ordenada. Essa nova sequência tende a agrupar caracteres similares, favorecendo compressão.

5. **Identificação da Posição Original**:
   - **Descrição**: Determina a posição da sequência original (com o `$`) na lista ordenada de rotações, essencial para reverter a BWT corretamente.

6. **Reversão da BWT (Reconstrução da Sequência)**:
   - **Descrição**: Reconstrói a sequência original com base na BWT e na posição original. Para isso, constrói iterativamente a matriz de rotações até identificar a linha correspondente à sequência original e remove o terminador ao final.

Code for PWM

In [11]:
from typing import List, Union

def bwt_transform(seq: str) -> List[Union[str, int]]:
    """
    Executa a transformação de Burrows-Wheeler (BWT) numa sequência.

    Parâmetros:
        seq (str): A sequência de entrada (sem o terminador).

    Retorna:
        List[Union[str, int]]: Uma lista com:
            - a sequência original;
            - a sequência transformada BWT;
            - o índice da sequência original na lista de rotações ordenadas.
    """
    original = seq  # Guarda a original
    if '$' not in seq:
        seq += '$'

    rotacoes = [seq[i:] + seq[:i] for i in range(len(seq))]
    rotacoes.sort()
    bwt = ''.join(rot[-1] for rot in rotacoes)

    return [original, bwt] 


def bwt_reverse(bwt: str) -> str:
    """
    Reverte a transformação BWT e recupera a sequência original.

    Parâmetros:
        bwt (str): A sequência transformada pela BWT.
        original_index (int): Índice da sequência original na matriz ordenada.

    Retorna:
        str: A sequência original (sem o terminador).
    """
    n = len(bwt)
    table = [''] * n
    start_index = bwt.index("$")

    for _ in range(n):
        # Adiciona bwt como prefixo a cada linha da tabela
        table = sorted([bwt[i] + table[i] for i in range(n)])

    # A linha original é aquela no índice original_index
    original = table[start_index]

    # Remover o símbolo de fim '$' antes de devolver
    return original.rstrip('$')


resultado = bwt_transform("banana")
print(resultado)
original_recuperado = bwt_reverse(resultado[1])
print("Sequência recuperada:", original_recuperado)




['banana', 'annb$aa']
Sequência recuperada: banana
